In [4]:
%matplotlib inline
import os
import json
from pprint import pprint
import matplotlib.pyplot as plt

from twython import TwythonStreamer
from collections import Counter

import string
import nltk

In [ ]:
# Set trans table for NLP
puncs = string.punctuation
dgts = string.digits
table_p = str.maketrans(puncs, len(puncs) * " ")

# Create stopword Dict
stopwords = nltk.corpus.stopwords.words('english')

In [2]:
# Import Gene's TwythonStreamer 

# Acknowledgement: This code is adopted from the book "Data Science from Scratch"
# https://github.com/joelgrus/data-science-from-scratch/tree/master/first-edition
class MyStreamer(TwythonStreamer):

    # by default, this module will collect 1000 tweets
    target_tweet_count = 1000
    
    # overriding
    def on_success(self, data):
        if 'text' in data:      # check if the received data has key 'text'
            tweets.append(data) # append data to our list
            
        if len(tweets) % 100 == 1:
            print('\nNow we collected {} tweets!'.format(len(tweets)))
            print(len(tweets), data['text'])    # print the tweet text

        if len(tweets) >= self.target_tweet_count:  # if we have 1000 tweets
            self.disconnect()   # disconnect API connection using a method from TwythonStreamer

    # overriding
    def on_error(self, status_code, data):
        print(status_code)
        self.disconnect()

# global list to collect the tweets
tweets = []       

def collect_tweets(stream, keyword, count):
    global tweets
    tweets = [] # re-initialize tweets list
    stream.target_tweet_count = count
    stream.statuses.filter(track=keyword)
    return tweets


## Create Stream

In [6]:
#Import credentials from env vars
consumer_key = str(os.environ['twit_consumer_key'])
consumer_secret = str(os.environ['twit_consumer_secret'])
access_token = str(os.environ['twit_access_token'])
access_token_secret = str(os.environ['twit_access_token_secret'])

In [7]:
#Create Stream
stream = MyStreamer(consumer_key, consumer_secret, access_token, access_token_secret)
print(stream)

In [ ]:
# keyword = 'Boris Johnson'
# #keyword = ''
# print('My keywords is ' + keyword)

In [ ]:
# start collecting 5000 tweets with keyword
# tweets = collect_tweets(stream, keyword, 5000)
# print(len(tweets), 'tweets are collected')

In [ ]:
# with open('tweet_stream_{}_{}.json'.format(keyword, len(tweets)), 'w') as outfile:
#     json.dump(tweets, outfile, indent=4)

In [8]:
keyword = 'Brexit'
#keyword = ''
print('My keywords is ' + keyword)

My keywords is Brexit


In [9]:
# # start collecting 5000 tweets with keyword
#  tweets = collect_tweets(stream, keyword, 5000)
# print(len(tweets), 'tweets are collected')


Now we collected 1 tweets!
1 RT @nickreeves9876: If Brexit happens it will be the first time in over 300 years that a hostile foreign power has successfully manipulated…

Now we collected 101 tweets!
101 RT @ActionWalsall: I was pretty ambivalent about EU until Brexit fever started, I’m fully in favour of remaining at the top table and tryin…

Now we collected 201 tweets!
201 RT @nyeannebevan: Now this is an interesting little article from the brilliant @skwawkbox back in February...about (ahem) *Opinium* CEO Jam…

Now we collected 301 tweets!
301 @BrookMatthew Hi, just a reminder that Brexit has not happened yet. It is due to occur on the 29th March 2019

Now we collected 401 tweets!
401 RT @grahambsi: For heavens sake @UKLabour and @LibDems voters get over yourselves. Vote tactically or have years to regret it in Brexit Bri…

Now we collected 501 tweets!
501 RT @jeremycorbyn: As Prime Minister, I will be an honest broker and not campaign for either side in the final say referendum. 

5000 tweets are collected


In [10]:
# with open('tweet_stream_{}_{}.json'.format(keyword, len(tweets)), 'w') as outfile:
#     json.dump(tweets, outfile, indent=4)

In [13]:
# Open files 
with open('tweet_stream_Brexit_5000.json') as infile:
    brex_data = json.load(infile)
    
with open('tweet_stream_Boris Johnson_5000.json') as infile:
    bj_data = json.load(infile)

# Put both datsets into a list for easier handling
data = [brex_data, bj_data]

In [23]:
"""
Path to get values in the file for reference
full tweet: brex_data[x]['retweeted_status']['extended_tweet']['full_text']
hashtags: brex_data[x]['entities']['hashtags']

"""


{'contributors': None,
 'coordinates': None,
 'created_at': 'Sat Nov 23 21:51:13 +0000 2019',
 'entities': {'hashtags': [],
              'symbols': [],
              'urls': [],
              'user_mentions': [{'id': 746371177174679552,
                                 'id_str': '746371177174679552',
                                 'indices': [3, 18],
                                 'name': 'Nick Reeves - REGISTER TO VOTE - '
                                         '#ABTV #FBPE',
                                 'screen_name': 'nickreeves9876'}]},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 1198358365749030913,
 'id_str': '1198358365749030913',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'retweeted': Fa

In [28]:
pprint(brex_data[0]['retweeted_status']['extended_tweet']['full_text'])

('If Brexit happens it will be the first time in over 300 years that a hostile '
 'foreign power has successfully manipulated a major British policy decision. '
 'We are now a dysfunctional third world kind of polity with a corrupt and '
 'broken politics wide open to foreign interference.')


In [30]:
brex_data[x]['entities']['hashtags']

[]